# **Crawling PTA, Berita, dan Link Page Output**

# **1. Crawling pta.trunojoyo.ac.id**

**Library**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time

**Base url**

In [ ]:
Base_Url = "https://pta.trunojoyo.ac.id/c_search/byprod"

**Function**

In [ ]:
def get_max_page(prodi_id):
    url = f"{Base_Url}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    # Cari tombol >> (last page)
    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        # Pecah URL -> ambil angka terakhir
        max_page = int(href.split("/")[-1])
        return max_page

    # fallback kalau pagination tidak ada
    return 1

In [ ]:
print(get_max_page(10))

172


In [ ]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

**Function All Data**

In [ ]:
def pta_all():
    start_time = time.time()

    data = {
        "id": [],
        "penulis": [],
        "judul": [],
        "abstrak_id": [],
        "abstrak_en": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "prodi": []
    }

    total_prodi = 1
    total_pages = 0
    max_pages_dict = {}

    # hitung total halaman (untuk tiap prodi)
    for i in range(1, total_prodi + 1):
        max_page = get_max_page(i)
        max_pages_dict[i] = max_page
        total_pages += max_page

    for i in range(1, total_prodi + 1):
        max_page = max_pages_dict[i]
        for j in range(1, max_page + 1):
            url = f"{Base_Url}/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")

            for jurnal in jurnals:
                link_keluar = jurnal.select_one('a.gray.button')['href']

                # ambil ID dari link PTA (angka terakhir di URL)
                id_match = re.search(r"/detail/(\d+)", link_keluar)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link_keluar)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')

                judul = isi.select_one('a.title').text.strip()
                penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]
                pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]
                pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

                paragraf = isi.select('p[align="justify"]')
                abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["abstrak_id"].append(abstrak_id)
                data["abstrak_en"].append(abstrak_en)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["prodi"].append(prodi)

            # update progress bar per prodi
            print_progress(i, prodi, j, max_page)

        sys.stdout.write("\n")  # pindah baris setelah 1 prodi selesai

    # simpan ke CSV
    df = pd.DataFrame(data)
    df.to_csv("pta_all.csv", index=False, encoding="utf-8-sig")

    # hitung durasi
    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📈 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [ ]:
pta_all()

/usr/local/lib/python3.12/dist-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


[1] Ilmu Hukum - Page 284/284 [████████████████████] 100.00%


✅ Seluruh data berhasil dikumpulkan!
📈 Total entri: 1417
⏱️ Waktu eksekusi: 3 jam 3 menit 35 detik


,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,080111100012,Dyah Ayu Citra Seza,Implementasi Fungsi Legislasi Dewan Perwakilan...,ABSTRAK\r\n\r\n Implementasi Fungsi Legi...,ABSTRACT\r\n Implementation of Legislati...,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",Ilmu Hukum
1,080111100002,Maulina Nurlaily,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Ilmu Hukum
2,070111100060,Moh. Samsul Hidayat,Analisis Terhadap Kekosongan Hukum dalam Penga...,Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi...","Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Ilmu Hukum
3,090111100077,TOMMY ADITYA PARLINDUNGAN MARBUN,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Ilmu Hukum
4,070111200007,RICA YENA IMADHORA,TELAAH KRITIS TENTANG ALASAN HUKUM YANG DIGUN...,,,"Dr. DENI SBY, S. H., M. S.","SAIFUL ABDULLAH, S. H., M. H.",Ilmu Hukum
...,...,...,...,...,...,...,...,...
1412,150111100130,DEDY DORES,PENGKUALIFIKASIAN CHEATER SEBAGAI TINDAK PIDAN...,Abstrak\n Perbuatan cheater dalam melakukan ch...,Abstract\n The way of cheater did a cheat in o...,"Aris Hardinanto, S.H., M.H.",,Ilmu Hukum
1413,150111100258,Eko Supriadi,KUALIFIKASI TINDAK PIDANA ATAS PERBUATAN PELAK...,Peminjaman dana sistem online dilakukan oleh m...,The loan funds by online system are carried ou...,"Dr. Erma Rusdiana, S.H.,M.H",,Ilmu Hukum
1414,160111100136,Muslimatul Maghfirah,KEDUDUKAN HUKUM PEKERJA OUTSOURCING DI DINAS P...,Abstrak\nTenaga kerja merupakan setiap orang y...,Abstract\nLabors are those who can work to pro...,"Mishbahul Munir, S.H., M.Hum",,Ilmu Hukum
1415,160111100024,MOH WASIL SYAHRONI,STAGNANSI HUBUNGAN KELEMBAGAAN DAN KEWENANGAN ...,Skripsi ini bertujuan untuk menganalisis penti...,This thesis aims to analyze the stagnation of ...,"Dr. DENI SETYA BAGUS YUHERAWAN, S.H., M.S",,Ilmu Hukum


**Function All Data 5 pages**

In [ ]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

def pta():
    start_time = time.time()  # mulai hitung waktu

    data = {
        "id": [],
        "penulis": [],
        "judul": [],
        "abstrak id": [],
        "abstrak en": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "prodi": [],
    }

    for i in range(1, 42):  # jumlah prodi
        total_pages = 5  # jumlah page
        prodi_name = None

        for j in range(1, total_pages + 1):  # loop page
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")
            if not prodi_name:
                prodi_name = prodi

            for jurnal in jurnals:
                link = jurnal.select_one('a.gray.button')['href']

                # ambil ID dari link PTA
                id_match = re.search(r"/detail/(\d+)", link)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')

                # Judul
                judul = isi.select_one('a.title').text

                # Penulis
                penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]

                # Pembimbing Pertama
                pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]

                # Pembimbing Kedua
                pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

                # Abstrak
                paragraf = isi.select('p[align="justify"]')
                abstrak = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstract = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                # simpan data
                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["abstrak id"].append(abstrak)
                data["abstrak en"].append(abstract)
                data["prodi"].append(prodi)

            # update progress bar
            print_progress(i, prodi_name, j, total_pages)

    df = pd.DataFrame(data)
    df.to_csv("pta.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📈 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [ ]:
pta()

[1] Ilmu Hukum - Page 5/5 [████████████████████] 100.00%

[2] Teknologi Industri Pertanian - Page 5/5 [████████████████████] 100.00%

[3] Agribisnis - Page 5/5 [████████████████████] 100.00%

[4] Agroteknologi - Page 5/5 [████████████████████] 100.00%

[5] Ilmu Kelautan - Page 5/5 [████████████████████] 100.00%

[6] Ekonomi Pembangunan - Page 5/5 [████████████████████] 100.00%

[7] Manajemen - Page 5/5 [████████████████████] 100.00%

[8] Akuntansi - Page 5/5 [████████████████████] 100.00%

[9] Teknik Industri - Page 5/5 [████████████████████] 100.00%

[10] Teknik Informatika - Page 5/5 [████████████████████] 100.00%

[11] Manajemen Informatika - Page 5/5 [████████████████████] 100.00%

[12] Sosiologi - Page 5/5 [████████████████████] 100.00%

[13] Ilmu Komunikasi - Page 5/5 [████████████████████] 100.00%

[14] Psikologi - Page 5/5 [████████████████████] 100.00%

[15] Sastra Inggris - Page 5/5 [████████████████████] 100.00%

[16] Ekonomi Syariah - Page 5/5 [████████████████████] 100.00%

,id,penulis,judul,abstrak id,abstrak en,pembimbing_pertama,pembimbing_kedua,prodi
0,080111100012,Dyah Ayu Citra Seza,Implementasi Fungsi Legislasi Dewan Perwakilan...,ABSTRAK\r\n\r\n Implementasi Fungsi Legi...,ABSTRACT\r\n Implementation of Legislati...,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",Ilmu Hukum
1,080111100002,Maulina Nurlaily,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Ilmu Hukum
2,070111100060,Moh. Samsul Hidayat,Analisis Terhadap Kekosongan Hukum dalam Penga...,Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi...","Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Ilmu Hukum
3,090111100077,TOMMY ADITYA PARLINDUNGAN MARBUN,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Ilmu Hukum
4,070111200007,RICA YENA IMADHORA,TELAAH KRITIS TENTANG ALASAN HUKUM YANG DIGUN...,,,"Dr. DENI SBY, S. H., M. S.","SAIFUL ABDULLAH, S. H., M. H.",Ilmu Hukum
...,...,...,...,...,...,...,...,...
776,160281100013,"Lisa Sri rahmatullah, S. Sos. I",Dampak Sosial Ekonomi Pariwisata Religi Makam ...,Penelitian ini bertujuan untuk mengetahui baga...,The purpose of this study is to analyze the so...,"Dr. Diah Wahyuningsih, S.E., M.Si.","Dr. Eni Sri Rahayuningsih, S.E., M.E.",Magister Ilmu Ekonomi
777,160281100002,Indah Ainun Nikmah,Peranan Zakat Produktif Dalam Meningkatkan Eko...,Peranan Zakat Produktif dalam Meningkatkan Eko...,The Role of Productive Zakat in Improving Must...,"Dr. Kurniyati Indahsari, M.Si","Dr. Abdur Rahman, S.Ag. MEI",Magister Ilmu Ekonomi
778,170361100010,ahmad syaiful umam,KARAKTERISASI DAN KOLEKSI PLASMA NUTFAH UNTUK ...,Madura merupakan salah satu wilayah pemasok ko...,Madura is one of the regions supplying horticu...,"Dr. Ir. Gita Pawana, M.Si","Dr. Ir. Hj. SIti Fatimah, M.Si",Magister Pengelolaan Sumber Daya Alam
779,170361100001,Siti Holifah,PENGOLAHAN LIMBAH AIR REBUSAN IKAN TERI MENJAD...,Ikan Teri perlu penanganan serius pasca panen ...,Anchovy needs serious handling after harvest b...,"Dr.Apri Arisandi,S.Pi.,M.Si.","Dr.Ir.H.Asfan,MP.",Magister Pengelolaan Sumber Daya Alam


**Page & Output Link PTA**

In [ ]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

def pta_links():
    start_time = time.time()  # mulai hitung waktu

    data = {
        "no": [],
        "page": [],
        "link_keluar": []
    }

    no = 1  # nomor urut

    for i in range(1, 42):  # jumlah prodi
        total_pages = 5  # jumlah page
        prodi_name = None

        for j in range(1, total_pages + 1):  # loop page
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")
            if not prodi_name:
                prodi_name = prodi

            for jurnal in jurnals:
                link = jurnal.select_one('a.gray.button')['href']

                data["no"].append(no)
                data["page"].append(url)          # link page
                data["link_keluar"].append(link)  # link detail
                no += 1

            # update progress bar
            print_progress(i, prodi_name, j, total_pages)

    df = pd.DataFrame(data)
    df.to_csv("pta_links.csv", index=False)

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh link berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [ ]:
pta_links()

[1] Ilmu Hukum - Page 5/5 [████████████████████] 100.00%

[2] Teknologi Industri Pertanian - Page 5/5 [████████████████████] 100.00%

[3] Agribisnis - Page 5/5 [████████████████████] 100.00%

[4] Agroteknologi - Page 5/5 [████████████████████] 100.00%

[5] Ilmu Kelautan - Page 5/5 [████████████████████] 100.00%

[6] Ekonomi Pembangunan - Page 5/5 [████████████████████] 100.00%

[7] Manajemen - Page 5/5 [████████████████████] 100.00%

[8] Akuntansi - Page 5/5 [████████████████████] 100.00%

[9] Teknik Industri - Page 5/5 [████████████████████] 100.00%

[10] Teknik Informatika - Page 5/5 [████████████████████] 100.00%

[11] Manajemen Informatika - Page 5/5 [████████████████████] 100.00%

[12] Sosiologi - Page 5/5 [████████████████████] 100.00%

[13] Ilmu Komunikasi - Page 5/5 [████████████████████] 100.00%

[14] Psikologi - Page 5/5 [████████████████████] 100.00%

[15] Sastra Inggris - Page 5/5 [████████████████████] 100.00%

[16] Ekonomi Syariah - Page 5/5 [████████████████████] 100.00%

,no,page,link_keluar
0,1,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/080...
1,2,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/080...
2,3,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/070...
3,4,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/090...
4,5,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/070...
...,...,...,...
776,777,https://pta.trunojoyo.ac.id/c_search/byprod/36/2,https://pta.trunojoyo.ac.id/welcome/detail/160...
777,778,https://pta.trunojoyo.ac.id/c_search/byprod/36/2,https://pta.trunojoyo.ac.id/welcome/detail/160...
778,779,https://pta.trunojoyo.ac.id/c_search/byprod/37/1,https://pta.trunojoyo.ac.id/welcome/detail/170...
779,780,https://pta.trunojoyo.ac.id/c_search/byprod/37/1,https://pta.trunojoyo.ac.id/welcome/detail/170...


# **2. Crawling Berita www.cnnindonesia.com**

**Library**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, re, sys

**Function**

In [ ]:
# Progress bar manual
def print_progress(current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\rPage {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

In [ ]:
# Ambil isi berita CNN
def get_article_content(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(r.text, "html.parser")

    paragraphs = []
    content_div = soup.find("div", class_="detail-text")
    if content_div:
        for p in content_div.find_all("p"):
            text = p.get_text(strip=True)
            if text and not text.lower().startswith("baca juga"):
                paragraphs.append(text)
    return " ".join(paragraphs)

In [ ]:
# Scraping CNN Indonesia
def berita_cnn(pages=1):
    start_time = time.time()

    BASE_URL = "https://www.cnnindonesia.com/indeks?page={}"

    data = {
        "id_berita": [],
        "judul_berita": [],
        "isi_berita": [],
        "kategori_berita": []
    }

    counter = 1  # mulai id dari 1

    for page in range(1, pages+1):
        url = BASE_URL.format(page)
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(r.text, "html.parser")

        articles = soup.select("article a")
        for a in articles:
            link = a.get("href")
            title = a.get_text(strip=True)

            if not link or not title:
                continue
            if not link.startswith("https://www.cnnindonesia.com/"):
                continue

            # Ambil kategori dari URL
            try:
                kategori = link.split("/")[3]
            except:
                kategori = "unknown"

            try:
                content = get_article_content(link)
            except:
                content = ""

            data["id_berita"].append(counter)
            data["judul_berita"].append(title)
            data["isi_berita"].append(content)
            data["kategori_berita"].append(kategori)

            counter += 1  # naikkan id

        print_progress(page, pages)

    df = pd.DataFrame(data)
    df.to_csv("cnnindonesia_berita4.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📈 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

# Contoh pemanggilan
df_cnn = berita_cnn(pages=100)

Page 100/100 [████████████████████] 100.00%


✅ Seluruh data berhasil dikumpulkan!
📈 Total entri: 1000
⏱️ Waktu eksekusi: 0 jam 10 menit 32 detik


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, re, sys

In [ ]:
berita_path ="/content/drive/MyDrive/PPW/output/cnnindonesia_berita4.csv"
berita = pd.read_csv(berita_path, index_col="id_berita")

berita

,judul_berita,isi_berita,kategori_berita
id_berita,,,
1,"Dorong Suporter, Andre Onana Dilempar BotolOla...",Kiper Manchester UnitedAndre Onanadilempar bot...,olahraga
2,Militer Nepal Batasi Aktivitas Pascademo Gulin...,Militer Nepalmengambil kembali kendali atas Ka...,internasional
3,Dua Helikopter Disiagakan Selama MotoGP Mandal...,Basarnas akan menyiapkan dan menyiagakan dua h...,olahraga
4,"Lari, Rahasia Hidup Sehat untuk Tingkatkan Pro...",Tingginya tuntutan pekerjaan di era yang serba...,gaya-hidup
5,Lokasi Titik Jatuh Helikopter di Mimika Ditemu...,Kepala Badan Nasional Pencarian dan Pertolonga...,nasional
...,...,...,...
996,"7 FOTOFOTO: Rusia Bombardir Ukraina, Luncurkan...",NaN,internasional
997,"Hasil MotoGP Catalunya: Alex Jadi Pemenang, Ka...",Pembalap Gresini Racing Alex Marquez memenangk...,olahraga
998,"Prabowo: Tuntutan 17+8 Sebagian Masuk Akal, Se...",Presiden RIPrabowo Subiantomenyatakan sebagian...,nasional


**Page & Link Output Berita**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, sys

# progress bar
def print_progress(current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\rPage {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

# fungsi untuk kumpulkan link berita CNN Indonesia
def berita_links(pages=1):
    start_time = time.time()

    BASE_URL = "https://www.cnnindonesia.com/indeks?page={}"

    data = {
        "id_berita": [],
        "page": [],
        "link_keluar": []
    }

    counter = 1
    for page in range(1, pages+1):
        url = BASE_URL.format(page)
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(r.text, "html.parser")

        articles = soup.select("article a")
        for a in articles:
            link = a.get("href")

            if not link or not link.startswith("https://www.cnnindonesia.com/"):
                continue

            data["id_berita"].append(counter)
            data["page"].append(url)       # halaman indeks
            data["link_keluar"].append(link)  # link detail berita

            counter += 1

        print_progress(page, pages)

    df = pd.DataFrame(data)
    df.to_csv("cnnindonesia_links1.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    print("\n✅ Seluruh link berhasil dikumpulkan!")
    print(f"📈 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

# contoh pemanggilan: ambil 100 halaman pertama
df_links = berita_links(pages=100)


Page 100/100 [████████████████████] 100.00%


✅ Seluruh link berhasil dikumpulkan!
📈 Total entri: 1000
⏱️ Waktu eksekusi: 0 jam 1 menit 0 detik


In [ ]:
link_path = "/content/drive/MyDrive/PPW/output/cnnindonesia_links1.csv"
link = pd.read_csv(link_path, index_col="id_berita")

link

,page,link_keluar
id_berita,,
1,https://www.cnnindonesia.com/indeks?page=1,https://www.cnnindonesia.com/nasional/20250911...
2,https://www.cnnindonesia.com/indeks?page=1,https://www.cnnindonesia.com/olahraga/20250911...
3,https://www.cnnindonesia.com/indeks?page=1,https://www.cnnindonesia.com/internasional/202...
4,https://www.cnnindonesia.com/indeks?page=1,https://www.cnnindonesia.com/olahraga/20250910...
5,https://www.cnnindonesia.com/indeks?page=1,https://www.cnnindonesia.com/gaya-hidup/202509...
...,...,...
996,https://www.cnnindonesia.com/indeks?page=100,https://www.cnnindonesia.com/internasional/202...
997,https://www.cnnindonesia.com/indeks?page=100,https://www.cnnindonesia.com/olahraga/20250907...
998,https://www.cnnindonesia.com/indeks?page=100,https://www.cnnindonesia.com/nasional/20250907...
